## Limpando e manipulando dados após Web Scraping

In [1]:
import pandas as pd
import re

In [2]:
file = "datasets/imoveis.json"

df = pd.read_json(file, orient = 'records')

df.head()

,preco,dormitorios,area,vagas,bairro
0,[R$ 1.055.000],[3],[128],[3],"[Imóvel 106391 - , Apartamento, \n ..."
1,[R$ 1.112.000],[3],[136.34],[2],"[Imóvel 130624 - , Apartamento, \n ..."
2,[R$ 1.620.000],[3],[204.87],[3],"[Imóvel 192740 - , Apartamento, \n ..."
3,[R$ 1.190.000],[3],[241],[2],"[Imóvel 196741 - , Casa, \n ..."
4,[R$ 3.053.011],[4],[271.3],[4],"[Imóvel 183630 - , Apartamento, \n ..."


In [3]:
print('Dimensões do Dataframe:\n')
print(f'Linhas: {df.shape[0]}')
print(f'Colunas: {df.shape[1]}')

Dimensões do Dataframe:

Linhas: 600
Colunas: 5


In [4]:
df.dtypes

preco          object
dormitorios    object
area           object
vagas          object
bairro         object
dtype: object

In [5]:
df['preco'].sample(20)

2      [R$ 1.620.000]
63       [R$ 750.000]
91       [R$ 230.000]
563      [R$ 458.000]
100      [R$ 310.000]
109    [R$ 1.280.000]
170      [R$ 980.000]
239      [R$ 320.000]
562      [R$ 360.000]
304      [R$ 800.000]
79       [R$ 750.000]
231      [R$ 475.000]
62     [R$ 1.980.000]
587      [R$ 335.000]
535      [R$ 680.000]
454      [R$ 396.296]
61       [R$ 270.000]
154      [R$ 440.000]
280      [R$ 369.000]
350    [R$ 1.056.000]
Name: preco, dtype: object

In [6]:
df['preco'] = df['preco'].apply(lambda x : re.sub('\D', '', str(x))).astype(float)

In [7]:
df['preco'].sample(10)

456     676000.0
561     460000.0
36     1349000.0
155    1413999.0
158    1700000.0
215     299000.0
10      780000.0
280     369000.0
504     636000.0
75      495000.0
Name: preco, dtype: float64

In [8]:
df.dtypes

preco          float64
dormitorios     object
area            object
vagas           object
bairro          object
dtype: object

In [9]:
df.isnull().sum()

preco           0
dormitorios     8
area            0
vagas          21
bairro          0
dtype: int64

In [10]:
df['dormitorios'] = df['dormitorios'].fillna("0")

In [11]:
df['dormitorios'] = df['dormitorios'].apply(lambda x : re.sub('\D', '', str(x))).astype(int)

In [12]:
df['dormitorios'].value_counts()

3    319
2    184
1     52
4     34
0      8
6      2
5      1
Name: dormitorios, dtype: int64

In [13]:
df['area'] = df['area'].apply(lambda x : re.sub('\D', '', str(x))).astype(float)

In [14]:
df['area'].sample(20)

148    13897.0
271     4195.0
170      239.0
350      176.0
380     6162.0
16        64.0
357    10561.0
173       64.0
171    19889.0
93      7684.0
402       56.0
160     9526.0
443      215.0
157     5695.0
421     5301.0
383     6798.0
4       2713.0
553     5838.0
177     5064.0
375     8387.0
Name: area, dtype: float64

In [15]:
df['vagas'] = df['vagas'].fillna("0")

In [16]:
df['vagas'] = df['vagas'].apply(lambda x : re.sub('\D', '', str(x))).astype(int)

In [17]:
df['vagas'].value_counts()

1    264
2    231
3     45
4     30
0     21
6      6
5      2
8      1
Name: vagas, dtype: int64

In [18]:
df['bairro'].sample(10)

172    [Imóvel 196491 - , Apartamento, \n            ...
49     [Imóvel 139071 - , Loft, \n                   ...
320    [Imóvel 153277 - , Apartamento, \n            ...
269    [Imóvel 113836 - , Apartamento, \n            ...
383    [Imóvel 195567 - , Apartamento, \n            ...
211    [Imóvel 168407 - , Apartamento, \n            ...
176    [Imóvel 194500 - , Apartamento, \n            ...
230    [Imóvel 178174 - , Cobertura, \n              ...
574    [Imóvel 195211 - , Apartamento, \n            ...
324    [Imóvel 175050 - , Apartamento, \n            ...
Name: bairro, dtype: object

In [19]:
df['bairro'] = df['bairro'].apply(lambda x : re.sub('\d', '', str(x))).astype(str)

In [20]:
df['bairro'].sample(10)

506    ['Imóvel  - ', 'Apartamento', '\n             ...
31     ['Imóvel  - ', 'Cobertura', '\n               ...
38     ['Imóvel  - ', 'Apartamento', '\n             ...
223    ['Imóvel  - ', 'Apartamento', '\n             ...
226    ['Imóvel  - ', 'Casa', '\n                    ...
256    ['Imóvel  - ', 'Apartamento', '\n             ...
213    ['Imóvel  - ', 'Casa', '\n                    ...
472    ['Imóvel  - ', 'Casa', '\n                    ...
145    ['Imóvel  - ', 'Apartamento', '\n             ...
108    ['Imóvel  - ', 'Apartamento', '\n             ...
Name: bairro, dtype: object

In [21]:
df['bairro'] = df['bairro'].apply(lambda x : re.sub('Imóvel  -', '', x))

In [22]:
df['bairro'] = df['bairro'].apply(lambda x : re.sub(r'\\n', '', x))

In [23]:
df['bairro'].sample(10)

55     [' ', 'Apartamento', '                        ...
133    [' ', 'Apartamento', '                        ...
574    [' ', 'Apartamento', '                        ...
233    [' ', 'Apartamento', '                        ...
503    [' ', 'Apartamento', '                        ...
95     [' ', 'Apartamento', '                        ...
255    [' ', 'Cobertura', '                        Pe...
522    [' ', 'Casa', '                        Sarandi...
46     [' ', 'Apartamento', '                        ...
120    [' ', 'Apartamento', '                        ...
Name: bairro, dtype: object

In [24]:
df['tipo'] = df['bairro'].apply(lambda x : re.split(',', x)[1])

In [25]:
df['tipo'].sample(10)

223     'Apartamento'
599     'Apartamento'
527            'Casa'
144            'Casa'
201     'Apartamento'
123            'Casa'
216     'Apartamento'
525     'Apartamento'
94             'Loft'
287     'Apartamento'
Name: tipo, dtype: object

In [26]:
df['bairro'] = df['bairro'].apply(lambda x : re.split(',', x)[2])

In [27]:
df['bairro']

0       '                        Central Parque
1          '                        Menino Deus
2           '                        Rio Branco
3              '                        Sarandi
4           '                        Bela Vista
                         ...                   
595        '                        Auxiliadora
596           '                        Tristeza
597         '                        Bela Vista
598      '                        Jardim Europa
599     '                        Três Figueiras
Name: bairro, Length: 600, dtype: object

In [28]:
df['bairro'] = df['bairro'].apply(lambda x : re.sub('\s', '', x))

In [29]:
df['bairro']

0      'CentralParque
1         'MeninoDeus
2          'RioBranco
3            'Sarandi
4          'BelaVista
            ...      
595      'Auxiliadora
596         'Tristeza
597        'BelaVista
598     'JardimEuropa
599    'TrêsFigueiras
Name: bairro, Length: 600, dtype: object

In [30]:
df['tipo'] = df['bairro'].apply(lambda x : re.sub("'", "", x))
df['bairro'] = df['bairro'].apply(lambda x : re.sub("'", '', x))

In [31]:
df.nunique()

preco          366
dormitorios      7
area           490
vagas            8
bairro          72
tipo            72
dtype: int64

In [32]:
df['tipo'].value_counts()

Petrópolis          46
PassodaAreia        41
MeninoDeus          34
BelaVista           24
Tristeza            23
                    ..
Navegantes           1
JardimLeopoldina     1
AbertadosMorros      1
ParqueSantaFé        1
Hipica               1
Name: tipo, Length: 72, dtype: int64

In [33]:
df.describe()

,preco,dormitorios,area,vagas
count,6.000000e+02,600.000000,600.000000,600.000000
mean,8.636065e+05,2.550000,8829.695000,1.725000
std,9.296298e+05,0.815644,14787.265011,1.015428
min,1.178000e+05,0.000000,42.000000,0.000000
25%,3.800000e+05,2.000000,685.750000,1.000000
50%,5.870000e+05,3.000000,6530.000000,2.000000
75%,9.710585e+05,3.000000,10249.000000,2.000000
max,9.117117e+06,6.000000,229098.000000,8.000000


In [34]:
df.groupby(by = 'bairro').preco.mean().sort_values(ascending = False)[:15]

bairro
MontSerrat          2.213700e+06
BelaVista           2.204248e+06
BelémNovo           2.134000e+06
PedraRedonda        1.950000e+06
JardimSãoPedro      1.900000e+06
TrêsFigueiras       1.667200e+06
Auxiliadora         1.588450e+06
JardimEuropa        1.455259e+06
CentralParque       1.244618e+06
MoinhosdeVento      1.242944e+06
ChácaradasPedras    1.221700e+06
RioBranco           1.190297e+06
Petrópolis          1.170219e+06
Agronomia           9.677143e+05
MeninoDeus          9.590551e+05
Name: preco, dtype: float64

In [35]:
df.loc[df['bairro'] == "BelaVista"]

,preco,dormitorios,area,vagas,bairro,tipo
4,3053011.0,4,2713.0,4,BelaVista,BelaVista
36,1349000.0,3,140.0,2,BelaVista,BelaVista
88,1699000.0,3,14015.0,2,BelaVista,BelaVista
114,1990000.0,3,18121.0,4,BelaVista,BelaVista
120,779000.0,2,8337.0,2,BelaVista,BelaVista
135,1400000.0,2,9904.0,2,BelaVista,BelaVista
159,540000.0,1,649.0,2,BelaVista,BelaVista
166,3420000.0,4,22602.0,4,BelaVista,BelaVista
188,3896550.0,3,51954.0,4,BelaVista,BelaVista
191,1699000.0,3,25062.0,2,BelaVista,BelaVista


In [36]:
df.to_csv('datasets/clean_imoveis.csv', index = False)